# Basis Selection for Highly-Converged Ground State

Alex's written-up notes for selecting an optimal set of ground state basis functions.


**Step 1. Number of l-channels**
Add l channels according to how heavy the element is
TODO(Alex). Need some simple rule of thumb for this, based on the atomic number.

**Step 2. MT Radii Optimisation**
Select muffin tin radii, according to an optimal ratio
Run the calculation for some trial energies, with lo recommendations on (rgkmax = 6-7, k-sampling ~ (4,4,4))
Look at RHO1D.XML and ensure the minimum of the density is not inside either MT radii.
If the largest MT radius is still far from the charge minimum, increase it (and increase the others such that the ratios between MT radii are preserved)

**Step 3. Selecting Core and Valence LOs**
Look at the isolated atomic energies in atoms.xml. This indicates which LOs should be in the core and valence, respectively.
For anything over >= -5 Ha, move to the valence.

```xml
  <atom chemicalSymbol="Ti" species="Ti.xml">
    <NumericalSetup TotalNumberOfGridPoints="1457" NumberOfMTGridPoints="600" GridType="cubic" rmin="1.000000000000e-6" rmt="1.800000000000e0" rmax="2.584376949949e1"/>
    <spectrum>
      <state n="1" l="0" kappa="1" energy="-178.425641565"/>
      <state n="2" l="0" kappa="1" energy="-19.6722601056"/>
      <state n="2" l="1" kappa="1" energy="-16.4511613208"/>
      <state n="2" l="1" kappa="2" energy="-16.2396076868"/>
      <state n="3" l="0" kappa="1" energy="-2.28715446689"/>   # Everthing from and including this function, so have core=false in the species file
      <state n="3" l="1" kappa="1" energy="-1.44088170217"/>
      <state n="3" l="1" kappa="2" energy="-1.41643304391"/>
      <state n="3" l="2" kappa="2" energy="-0.164604761314"/>
      <state n="4" l="0" kappa="1" energy="-0.168715959560"/>
    </spectrum>
  </atom>
```

Set them to core = false in species file.

**Step 4. Matching Orders of LOs**

A basic pattern is:

```xml
      <custom l="1" type="lapw" trialEnergy="-0.9" searchE="false"/>
      <lo l="1">
        <wf matchingOrder="0" trialEnergy="-0.9" searchE="false"/>
        <wf matchingOrder="1" trialEnergy="-0.9" searchE="false"/>
      </lo>
      <lo l="1">
        <wf matchingOrder="1" trialEnergy="-0.9" searchE="false"/>
        <wf matchingOrder="2" trialEnergy="-0.9" searchE="false"/>
      </lo>
```

TODO(Alex). Explain why this is equally valid
however this can equivalently be:

```xml
      <custom l="1" type="lapw" trialEnergy="-0.9" searchE="false"/>
      <lo l="1">
        <wf matchingOrder="0" trialEnergy="-0.9" searchE="false"/>
        <wf matchingOrder="1" trialEnergy="-0.9" searchE="false"/>
      </lo>
      <lo l="1">
        <wf matchingOrder="0" trialEnergy="-0.9" searchE="false"/>
        <wf matchingOrder="2" trialEnergy="-0.9" searchE="false"/>
      </lo>
```

For any l-channel in the valence that has more than one pqn for example 3s and 4s, one needs to add an lo with matching orders (0, 0) for the radial functions, and trial energies equal to 3s and 4s
In lo recommendations, an SCF calculation is done for the crystal and the density in a MT region is then spherically averaged and used in the solution of the radial Schrodinger equation to obtain
optimal LO recommendations for the linearisation energies.

LO recommendations are only listed w.r.t. to nodes (n is misleading. n means nodes in LOrecommendations and pqn in the species file).
One can label as

node = pqn - l - 1
=>
pqn = node + (l + 1)
```
 Energy parameters
 ------------
 species           1
 l=           0
 n=           0  -179.355067856822     1s
 n=           1  -19.3109495266249     2s
 n=           2  -1.83220324630948     3s
 n=           3   3.48339510536654     4s
 n=           4   12.9105889690604     5s
 n=           5   26.1707941233132     6s
 n=           6   42.9401549493363     7s

 l=           1
 n=           0  -15.9111558662072     2p
 n=           1 -0.923625584149692     3p
 n=           2   4.04364738034662     4p
 n=           3   13.0643199063681     5p
 n=           4   25.6645524508287     6p
 n=           5   41.6099089096687     7p
 n=           6   60.8089462978516     9p
 ...
```

so for the basis:


```
    <atomicState n="1" l="0" kappa="1" occ="2.00000" core="true"/>
    <atomicState n="2" l="0" kappa="1" occ="2.00000" core="true"/>
    <atomicState n="2" l="1" kappa="1" occ="2.00000" core="true"/>
    <atomicState n="2" l="1" kappa="2" occ="4.00000" core="true"/>
    <atomicState n="3" l="0" kappa="1" occ="2.00000" core="false"/>
    <atomicState n="3" l="1" kappa="1" occ="2.00000" core="false"/>
    <atomicState n="3" l="1" kappa="2" occ="4.00000" core="false"/>
    <atomicState n="3" l="2" kappa="2" occ="2.00000" core="false"/>
    <atomicState n="4" l="0" kappa="1" occ="2.00000" core="false"/>
```
both the 4s and 3s (same l, different pqn) are in the valence. One needs to set an LO

```
      <custom l="0" type="lapw" trialEnergy="-1.8322" searchE="false"/>
      <lo l="0">
        <wf matchingOrder="0" trialEnergy="-1.8322" searchE="false"/>    Energy corresponds to function with nodal structure of the 3s
        <wf matchingOrder="1" trialEnergy="-1.8322" searchE="false"/>    Energy corresponds to function with nodal structure of the 3s
      </lo>
      <lo l="0">
        <wf matchingOrder="1" trialEnergy="-1.8322" searchE="false"/>    Energy corresponds to function with nodal structure of the 3s
        <wf matchingOrder="2" trialEnergy="-1.8322" searchE="false"/>    Energy corresponds to function with nodal structure of the 3s
      </lo>
      <lo l="0">
        <wf matchingOrder="0" trialEnergy="-1.8322" searchE="false"/>   Energy corresponds to function with nodal structure of the 3s
        <wf matchingOrder="0" trialEnergy="3.4834" searchE="false"/>    Energy corresponds to function with nodal structure of the 4s
      </lo>
```
where the last LO ensures one can have separate basis solutions for the 3s and 4s
TODO(Alex) Show this mathematically.


**Step 5. Evaluate Core Leakage**
Core leakage can be reduced by increasing MT radius, or by moving core to valence.
The deeper the core state, the more atomic-like hence tightly-bound it is.


**Step 6. Converge W.r.t. k-grid and rgkmax**

